In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 53 (delta 20), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [ ]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [ ]:
! bash install_mecab-ko_on_colab190912.sh
from konlpy.tag import *
mecab = Mecab()

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 2.3MB 44.7MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-02-06 18:08:46--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.2, 18.205.93.1, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
ver = input(str('버전입력'))
path = '/content/gdrive/My Drive/sichu'

Mounted at /content/gdrive
버전입력review_0206


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
def tokenizer(x):
    return mecab.morphs(x)
def tokenizer2(x):
    return mecab.nouns(x)

import pickle
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import namedtuple
import time
from tqdm import *

doc_vec_lecture = Doc2Vec(
    dm=1,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    size=200,        # vector size
    alpha=0.025,     # learning-rate
    seed=777,
    min_count=50,    # ignore with freq lower
    min_alpha=0.015, # min learning-rate
    workers=8)   # multi cpu
    #hs = 1,          # hierarchical softmax / default 0
    #negative = 10,   # negative sampling / default 5)
    
doc_vec_prof = Doc2Vec(
    dm=1,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    size=200,        # vector size
    alpha=0.025,     # learning-rate
    seed=777,
    min_count=50,    # ignore with freq lower
    min_alpha=0.015, # min learning-rate
    workers=8)   # multi cpu
    #hs = 1,          # hierarchical softmax / default 0
    #negative = 10,   # negative sampling / default 5)

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [ ]:
class doc2vec(object):
    
    def __init__(self):
        self.data=pd.read_csv(path+'/'+ver+'.csv')
    
    def make_lecture(self,min_count = 10):
        
        review = self.data
        keys=[]
        token = review.groupby(['name','prof'])['review'].agg(lambda x : ' '.join(x)).reset_index()
        token['token'] = token['review'].apply(tokenizer)
        
        for n,f in zip(token['name'],token['prof']):
            keys.append('-'.join([n,f]))
        
        token['keys']=keys

        self.token_lecture = token
        
        
        TaggedDocument = namedtuple('TaggedDocument', 'words tags')
        tagged_train_docs = [TaggedDocument(d,c) for d, c in token[['token', 'keys']].values]
        doc_vec_lecture.build_vocab(tagged_train_docs)
        
        for epoch in range(20):
            doc_vec_lecture.train(tagged_train_docs, total_examples=doc_vec_lecture.corpus_count, epochs=doc_vec_lecture.iter)
            doc_vec_lecture.alpha -= 0.002 # decrease the learning rate
            doc_vec_lecture.min_alpha = doc_vec_lecture.alpha # fix the learning rate, no decay

        X = np.array([doc_vec_lecture.infer_vector(doc.words) for doc in tagged_train_docs])
        y = np.array([doc.tags for doc in tagged_train_docs])
        X_sparse = sparse.csr_matrix(X)
        self.similarity_lecture = cosine_similarity(X_sparse)
        self.tags_lecture = y
        sim = self.similarity_lecture
        tags = self.tags_lecture
        token = self.token_lecture

        d2v_lec={}
        for i in tqdm_notebook(range(len(tags))) :
          sim_idx = np.argsort(sim[i])[::-1][1:4]
          values=[]
          for idx in sim_idx:
            values.append(tags[idx])
          d2v_lec[tags[i]] = values
        self.d2v_lec_dict = d2v_lec
        
    def make_prof(self,min_count = 10):

        review = self.data
        keys=[]
        
        token = review.groupby(['prof'])['review'].agg(lambda x : ' '.join(x)).reset_index()
        token['token'] = token['review'].apply(tokenizer)
        token['keys'] = token['prof']

        self.token_prof = token
        
        
        TaggedDocument = namedtuple('TaggedDocument', 'words tags')
        tagged_train_docs = [TaggedDocument(d,c) for d, c in token[['token', 'keys']].values]
        doc_vec_prof.build_vocab(tagged_train_docs)
        
        for epoch in range(20):
            doc_vec_prof.train(tagged_train_docs, total_examples=doc_vec_prof.corpus_count, epochs=doc_vec_prof.iter)
            doc_vec_prof.alpha -= 0.002 # decrease the learning rate
            doc_vec_prof.min_alpha = doc_vec_prof.alpha # fix the learning rate, no decay

        X = np.array([doc_vec_prof.infer_vector(doc.words) for doc in tagged_train_docs])
        y = np.array([doc.tags for doc in tagged_train_docs])
        
        X_sparse = sparse.csr_matrix(X)
        self.similarity_prof = cosine_similarity(X_sparse)
        self.tags_prof = y
        sim = self.similarity_prof
        tags = self.tags_prof
        token = self.token_prof

        d2v_prof={}
        for i in tqdm_notebook(range(len(tags))) :
          sim_idx = np.argsort(sim[i])[::-1][1:4]
          values=[]
          for idx in sim_idx:
            values.append(tags[idx])
          d2v_prof[tags[i]] = values
        self.d2v_prof_dict = d2v_prof        

In [ ]:
class tf(object) :
    
    def __init__(self):
        self.data=pd.read_csv(path+'/'+ver+'.csv')
        self.stop_words=['중간','기말','교수','수업','강의','수강','말','사람','수준','안','앞','왜','이','자기','조금','좀','주시','주심','중','처음','문제','시험','기말고사','중간고사'
                    '크게','것','진짜','학점','방식','각', '간', '감', '갑', '값', '갓', '강', '개', '갠', '갯', '갸', '거', '건', '걸', '겁', '겉', '게', '겨', '결', '겸', '경', '고', '골', '곳', '공', '과', '관', '교', '구', '국', '굳', '굿', '권', '귀', '규', '그', '극', '글', '금', '급', '기', '길', '김', '깐', '꺼', '껀', '꼴', '꽁', '꽃', 
                     '꿈', '끝', '나', '난', '날', '남', '낭', '내', '냄', '너', '년', '노', '놈', '뇌', '눈', '느', '니', '님', '닝', '다', '닥', '달', '담', '답', '당', '대', '덕', '덤', '데', '도', '독', '돈', '동', '둘', '뒤', '드', '듬', '듯', '등', '디', '딥', '딩', '땀', '때', '땐', '땜', '똥', '뜸', '뜻', '띠', '랍', '랑', '랩', '랫', 
                    '량', '렸', '로', '르', '를', '리', '림', '막', '만', '맘', '맛', '망', '매', '맥', '머', '면', '명', '몇', '모', '목', '몸', '못', '문', '물', '뭐', '뭘', '뭣', '뮤', '미', '밋', '밑', '바', '박', '밖', '반', '발', '밤', '밥', '방', '배', '백', '버', '번', '법', '벽', '별', '복', '본', '볼', '봄', '부', '북', '분', '불', 
                    '브', '비', '빅', '빛', '빠', '빵', '뻔', '뿐', '뿔', '사', '산', '살', '삶', '삼', '상', '샷', '선', '설', '성', '세', '셀', '셈', '셉', '셋', '션', '셜', '셤', '속', '손', '솔', '수', '순', '술', '숨', '쉽', '슈', '승', '시', '식', '신', '실', '심', '십', '쌤', '쏘', '쓰', '씨', '씹', '악', '알', '암', '애', '앵', '양', '얘', '엄', '업', '엠', '여', '역', '열', '엿', '영', '옆', 
                    '예', '오', '옷', '왕', '외', '욕', '욤', '욥', '용', '우', '운', '워', '원', '월', '위', '유', '윤', '융', '은', '음', '의', '인', '일', '임', '입', '잉', '자', '잠', '장', '잼', '저', '적', '전', '절', '점', '정', '제', '조', '존', '종', '주', '준', '줄', '줌', '쥬', '쥴', '지', '직', '진', '질', '짐', '집', '짓', '짝', '짬', '짱', '쪽', '쯤', '차', '참', '창', '채', '책', '척', 
                    '천', '철', '첨', '청', '체', '첵', '초', '총', '추', '축', '춤', '충', '취', '칠', '침', '카', '칸', '칼', '컨', '컴', '컷', '코', '쿨', '큐', '키', '타', '탈', '탑', '탓', '택', '터', '테', '토', '톡', '톤', '통', '투', '툴', '튀', '트', '틀', '틈', '티', '틱', '팁', '판', '팔', '패', '팩', '퍼', '펙', '편', '평', '포', '폭', '폰', '표', '풀', '품', '플', '피', '하', '학',
                     '한', '할', '합', '해', '핵', '헬', '현', '형', '호', '혼', '홍', '화', '회', '후', '흠', '히', '힘','공부','학생','성적','전주','이수업','과목','비플','이해','주제','클','내용'
                     ]

        
    def make_lecture(self,min_df=10):
        review = self.data
        self.tf_data_lecture = review.groupby(['name','prof'])['review'].agg(lambda x : ' '.join(x)).reset_index()
        self.corpus_lecture = self.tf_data_lecture['review'].tolist()
        
        tok_corpus=[]
        for doc in self.corpus_lecture:
            tok_corpus.append(tokenizer(doc))
        self.tok_corpus_lecture = tok_corpus
        
        self.tfidf_lecture = TfidfVectorizer(tokenizer=tokenizer, min_df=min_df).fit(self.corpus_lecture)
        self.tfidf_matrix_lecture = self.tfidf_lecture.transform(self.corpus_lecture).toarray()
        
        self.tfidf_tag_lecture = TfidfVectorizer(tokenizer=tokenizer2, min_df=10,stop_words=self.stop_words).fit(self.corpus_lecture)
        self.tfidf_matrix_tag_lecture = self.tfidf_tag_lecture.transform(self.corpus_lecture).toarray()
        
        self.cos_mat_lecture=cosine_similarity(sparse.csr_matrix(self.tfidf_matrix_lecture))
        self.hash_lecture = self.tfidf_tag_lecture.get_feature_names()
        
        ######################
        
        tf_data = self.tf_data_lecture
        cos_mat = self.cos_mat_lecture
        
        idx_dict={}
        for i in range(len(tf_data)):
          idx_dict[i] = '-'.join(tf_data.iloc[i,:2])

        tdf_lec={}
        for i in tqdm_notebook(range(len(tf_data))):
          jac_sim={}
          sim_idx = np.argsort(cos_mat[i,:])[::-1][1:50]
          
          for idxs in sim_idx:
            union = set(tok_corpus[i]).union(set(tok_corpus[idxs]))
            intersection = set(tok_corpus[i]).intersection(set(tok_corpus[idxs]))
            jac_sim[idxs] = len(intersection)/len(union)
            out_idx = list(pd.Series(jac_sim).sort_values(ascending=False)[:8].index)

            values=[]

            for keys in out_idx:
              values.append(idx_dict.get(keys))
              
              tdf_lec[idx_dict.get(i)]=values

        self.tf_lec_dict = tdf_lec

        hashs = self.hash_lecture
        tfidf_matrix_tag = self.tfidf_matrix_tag_lecture
        lecture_tags={}
        for name,prof in tqdm_notebook(zip(tf_data['name'],tf_data['prof'])):
            idx = tf_data.query('name == @name & prof == @prof').index[0]
            nouns = np.argsort(tfidf_matrix_tag[idx,:])[::-1][:5]
            hash_tags=[]
            for hash_tag in nouns :
              hash_tags.append(hashs[hash_tag])
            lecture_tags[(name,prof)] = hash_tags
        self.tf_lec_tags = lecture_tags

        ######################
        
          
    def make_prof(self,min_df=10):
        review = self.data
        self.tf_data_prof = review.groupby('prof')['review'].agg(lambda x : ' '.join(x)).reset_index()
        self.corpus_prof = self.tf_data_prof['review'].tolist()
        
        tok_corpus=[]
        for doc in self.corpus_prof:
            tok_corpus.append(tokenizer(doc))
        self.tok_corpus_prof = tok_corpus
        
        self.tfidf_prof = TfidfVectorizer(tokenizer=tokenizer, min_df=min_df).fit(self.corpus_prof)
        self.tfidf_matrix_prof = self.tfidf_prof.transform(self.corpus_prof).toarray()
        
        self.tfidf_tag_prof = TfidfVectorizer(tokenizer=tokenizer2, min_df=10,stop_words=self.stop_words).fit(self.corpus_prof)
        self.tfidf_matrix_tag_prof = self.tfidf_tag_prof.transform(self.corpus_prof).toarray()
        
        self.cos_mat_prof = cosine_similarity(sparse.csr_matrix(self.tfidf_matrix_prof))
        self.hash_prof = self.tfidf_tag_prof.get_feature_names()

        #############################
        tf_data = self.tf_data_prof
        cos_mat = self.cos_mat_prof
        tok_corpus = self.tok_corpus_prof

        idx_dict={}
        for i in (range(len(tf_data))):
          idx_dict[i] = (tf_data.iloc[i,0])

        tdf_prof={}
        for i in tqdm_notebook(range(len(tf_data))):
          jac_sim={}
          sim_idx = np.argsort(cos_mat[i,:])[::-1][1:50]
    
          for idxs in sim_idx:
              compute_ = tok_corpus[idxs]
              union = set(tok_corpus[i]).union(set(tok_corpus[idxs]))
              intersection = set(tok_corpus[i]).intersection(set(tok_corpus[idxs]))
              jac_sim[idxs] = len(intersection)/len(union)
              out_idx = list(pd.Series(jac_sim).sort_values(ascending=False)[:8].index)
              
              values=[]
              
              for keys in out_idx:
                  values.append(idx_dict.get(keys))
                  tdf_prof[idx_dict.get(i)]=values

        self.tf_prof_dict = tdf_prof

        tf_data = self.tf_data_prof
        cos_mat = self.cos_mat_prof
        hashs = self.hash_prof
        tfidf_matrix_tag = self.tfidf_matrix_tag_prof
        tok_corpus = self.tok_corpus_prof
        prof_tags={}
        for prof in tf_data['prof']:
            idx = tf_data.query('prof == @prof').index[0]
            nouns = np.argsort(tfidf_matrix_tag[idx,:])[::-1][:5]
            hash_tags=[]
            for hash_tag in nouns :
              hash_tags.append(hashs[hash_tag])
            prof_tags[prof] = hash_tags
        self.tf_prof_tags = prof_tags

In [ ]:
d2v = doc2vec()
d2v.make_lecture()
d2v.make_prof()

tfidf = tf()
tfidf.make_lecture()
tfidf.make_prof()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['중간고사'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['중간고사'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
# Write
path = '/content/gdrive/My Drive/sichu/'
with open(path+'get_tf.pickle', 'wb') as f:
            pickle.dump(get_tf, f, pickle.HIGHEST_PROTOCOL)
with open(path+'get_d2v.pickle', 'wb') as f:
            pickle.dump(d2v, f, pickle.HIGHEST_PROTOCOL)

NameError: ignored

In [ ]:
# Load
path = '/content/gdrive/My Drive/'

with open (path+'get_tf.pickle','rb') as d :
    get_tf = pickle.load(d)

with open (path+'get_d2v.pickle','rb') as d :
    d2v = pickle.load(d)

In [ ]:
lst=[]

for a,b in zip(tfidf.tf_lec_dict.values(),d2v.d2v_lec_dict.values()):
  lst.append(list(set(a+b)))

lec = pd.DataFrame(d2v.d2v_lec_dict.keys(),columns=['key'])
lec['sim']=pd.Series(list(lst))
lec['tags']=pd.Series(list(tfidf.tf_lec_tags.values()))

lst=[]

for a,b in zip(tfidf.tf_prof_dict.values(),d2v.d2v_prof_dict.values()):
  lst.append(list(set(a+b)))

prof = pd.DataFrame(d2v.d2v_prof_dict.keys(),columns=['key'])
prof['sim']=pd.Series(list(lst))
prof['tags']=pd.Series(list(tfidf.tf_prof_tags.values()))

In [ ]:
lec.to_pickle(path+'lec_sim.pkl')
prof.to_pickle(path+'prof_sim.pkl')